In [1]:
!pip install pycaret
import pandas as pd
import numpy as np
from pycaret.time_series import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 87.1 MB/s eta 0

In [2]:
from google.colab import files
import io
uploaded = files.upload()
data_file = pd.read_csv(io.BytesIO(uploaded["AAPL(Apple) stock.csv"]))
data_file.head()

Saving AAPL(Apple) stock.csv to AAPL(Apple) stock.csv


,Date,Close,Volume
0,2013-01-02,19.608213,560518000
1,2013-01-03,19.360714,352965200
2,2013-01-04,18.821428,594333600
3,2013-01-07,18.710714,484156400
4,2013-01-08,18.761070,458707200


In [4]:
data_file["index"] = pd.to_datetime(data_file["Date"])
data_file = data_file.drop(["Date"], axis = 1)
data_file = data_file.set_index("index")
data_file.index = data_file.index.to_period('D')
data_file.head()


,Close,Volume
index,,
2013-01-02,19.608213,560518000
2013-01-03,19.360714,352965200
2013-01-04,18.821428,594333600
2013-01-07,18.710714,484156400
2013-01-08,18.761070,458707200


In [5]:
import numpy as np
complete_index = pd.period_range(start=min(data_file.index), end=max(data_file.index), freq='D')
data_file = data_file.reindex(complete_index, fill_value=np.nan)
data_file.head()

,Close,Volume
2013-01-02,19.608213,560518000.0
2013-01-03,19.360714,352965200.0
2013-01-04,18.821428,594333600.0
2013-01-05,NaN,NaN
2013-01-06,NaN,NaN


In [6]:
mean_value = data_file['Close'].mean()
data_file['Close'].fillna(value=mean_value, inplace=True)
mean_value = data_file['Volume'].mean()
data_file['Volume'].fillna(value=mean_value, inplace=True)
data_file.head()

,Close,Volume
2013-01-02,19.608213,5.605180e+08
2013-01-03,19.360714,3.529652e+08
2013-01-04,18.821428,5.943336e+08
2013-01-05,62.416599,1.713540e+08
2013-01-06,62.416599,1.713540e+08


In [8]:
setup = setup(data_file, fh = 12, fold = 3, target = 'Close')

,Description,Value
0,session_id,4928
1,Target,Close
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(3650, 2)"
5,Transformed data shape,"(3650, 2)"
6,Transformed train set shape,"(3638, 2)"
7,Transformed test set shape,"(12, 2)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [9]:
setup.plot_model()

In [10]:

setup.plot_model(plot = 'acf')

In [11]:

setup.plot_model(plot = 'pacf')

In [12]:

setup.plot_model(plot = 'periodogram')

In [13]:

setup.plot_model(plot = 'fft')

In [14]:

setup.plot_model(plot = 'diagnostics')

In [15]:
best_model = setup.compare_models()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,2.1824,1.0049,6.7205,10.6255,0.0730,0.0606,0.8895,0.3667
arima,ARIMA,2.8715,1.7693,8.8556,18.7496,0.0826,0.0739,0.7337,1.0733
stlf,STLF,2.9214,1.5864,9.0213,16.8220,0.0873,0.0756,0.8068,0.1667
snaive,Seasonal Naive Forecaster,2.9703,1.7975,9.1599,19.0485,0.0842,0.0757,0.7246,0.8167
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,3.2889,1.7690,10.1562,18.7535,0.1023,0.0923,0.7250,0.1500
auto_arima,Auto ARIMA,3.3442,1.5841,10.3179,16.7861,0.0979,0.0894,0.7841,86.1433
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,3.6890,1.4668,11.3816,15.5518,0.1165,0.1080,0.8322,0.4000
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,4.2216,1.9624,13.0429,20.8187,0.1659,0.1292,0.6527,78.0867
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,4.2788,1.8489,13.2030,19.5954,0.1381,0.1235,0.7192,0.1367
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,4.2798,1.8490,13.2059,19.5963,0.1382,0.1236,0.7192,0.2133


Processing:   0%|          | 0/101 [00:00<?, ?it/s]

In [16]:
created_model = setup.create_model(best_model)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2022-11-12,4.1129,2.1447,12.6155,22.6042,0.1440,0.1088,0.6990
1,2022-11-24,1.6126,0.5484,4.9882,5.8287,0.0546,0.0528,0.9781
2,2022-12-06,0.8216,0.3216,2.5579,3.4435,0.0204,0.0203,0.9915
Mean,NaT,2.1824,1.0049,6.7205,10.6255,0.0730,0.0606,0.8895
SD,NaT,1.4027,0.8113,4.2848,8.5260,0.0521,0.0366,0.1348


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
predicted_model = setup.predict_model(created_model)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,ExtraTreesRegressor,3.1841,1.0401,9.9143,11.1226,0.0876,0.0822,0.8608


In [18]:
plot_model(created_model)

In [20]:
tuned_model = setup.tune_model(created_model)
plot_model(tuned_model)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2022-11-12,4.5870,2.4034,14.0697,25.3308,0.1865,0.1386,0.6221
1,2022-11-24,4.3934,1.9877,13.5896,21.1280,0.1534,0.1469,0.7126
2,2022-12-06,2.4807,0.8365,7.7230,8.9576,0.0931,0.0860,0.9422
Mean,NaT,3.8204,1.7425,11.7941,18.4721,0.1443,0.1238,0.7590
SD,NaT,0.9506,0.6628,2.8854,6.9431,0.0387,0.0269,0.1348


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   25.4s finished


In [21]:
saved_model =  setup.save_model(tuned_model, "finalized_model_time_series_without_exogenous_variables")

Transformation Pipeline and Model Successfully Saved


In [22]:
saved_model

(ForecastingPipeline(steps=[('forecaster',
                             TransformedTargetForecaster(steps=[('model',
                                                                 BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [7,
                                                                                                                                         6,
                                                                                                                                         5,
                                                                                                                                         4,
                                                                                                                                         3,
                                                                                                                                         2,
                                           